In [1]:
%matplotlib inline
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SearchIO
from pybedtools import BedTool
import numpy as np
import pybedtools
import multiprocessing
import re
import time
import matplotlib.pyplot as plt
import json
from ipywidgets import widgets
from IPython.display import display
from sklearn.externals.joblib import Parallel, delayed
import seaborn as sns
import matplotlib
import subprocess

/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [11]:
DK0911_genome_folder = '/home/benjamin/genome_assembly/Warrior/genome_v04'
DK0911_GFF_file = os.path.join(DK0911_genome_folder, 'DK_0911_v04_ph_ctg.anno.gff3')
DK0911_secretome_folder = '/home/benjamin/genome_assembly/Warrior/DK0911/Secretome'
DK0911_annotation_folder_pcontigs = '/home/benjamin/genome_assembly/Warrior/DK0911_v04/protein_annotation/DK_0911_v04LT_p_ctg/combined'
DK0911_annotation_folder_hcontigs = '/home/benjamin/genome_assembly/Warrior/DK0911_v04/protein_annotation/DK_0911_v04LT_h_ctg/combined'
DK0911_output_folder = '/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations'

In [30]:
#from https://github.com/BenjaminSchwessinger/AvrYr_hunting/blob/master/fisher/Pull_out_gene_beds_and_gffs.ipynb
#just a quick function to convert the gff locus tag to a bed file for later convinience
def gff_locus_tag_to_bed6(gff_fn, bed_fn, locID_list, gff=False):
    """
    This function converts a gff file with locus_tags to a bed 6 format using
    the locus tag id. The bed out file will be for genes only.
    Input: Filename form gff file.
           Bed out file name.
           list of locusID_tags for subsetting.
           If gff is True also put out the gff file. This is pretty slow for now.
    Output: File converted to bed. Same filename as gff file just ending in bed.
            And a subsetted gff file as well."""
    #now compile the pattern for search the locusTagID in the gff file.
    loc_ID_pattern = re.compile(r'locus_tag=([^;]*)')
    ID_pattern = re.compile(r'ID=(.*);Parent')
    #write test if directory of bed_fn exists
    #write function to check if bed_fn already exists
    
    #prepare for gff parsing as well
    ids_to_keep = []
    
    #lets handle the bed file first
    with open(gff_fn, 'r') as gff_fh:
        with open(bed_fn, 'w') as bed_fh:
            for line in gff_fh:
                line = line.rstrip()
                e = line.split('\t')
                if e[2] == 'gene':
                    loc_ID = re.findall(loc_ID_pattern, line)[0]
                    #this now subsects the outfile for members included in the locus_tag list
                    if loc_ID in locID_list:
                        print('%s\t%s\t%s\t%s\t%s\t%s' % (e[0], int(e[3]) - 1 , e[4], loc_ID, e[7], e[6]), file=bed_fh)
                #here we populate the list of ids we want to keep
                elif e[2] == 'mRNA':
                    loc_ID = re.findall(loc_ID_pattern, line)[0]
                    if loc_ID in locID_list:
                        ids_to_keep.append(re.findall(ID_pattern, line)[0])                    
                else:
                    continue
    if gff == True:
        with open(gff_fn, 'r') as gff_fh:
            with open(bed_fn.replace('.bed', '.gff3'), 'w') as gff_out_fh:
                for line in gff_fh:
                    line = line.rstrip()
                    e = line.split('\t')
                    if e[2] == 'gene':
                        loc_ID = re.findall(loc_ID_pattern, e[8])[0]
                        if loc_ID in locID_list:
                            print(line, file=gff_out_fh)   
                    elif any([re.search(r'\b%s\b' % x, e[8]) \
                            for x in ids_to_keep]):
                          print(line, file=gff_out_fh) 

                    else:
                        continue

### Example of output file formate Pst_104E_v13_ph_ctg.secretome.gene.bed

In [4]:
!ls /home/benjamin/genome_assembly/Warrior/DK0911/Secretome

DK_0911_v04_h_ctg.EffectorP2.0.fa  DK_0911_v04_p_ctg.protein.fa
DK_0911_v04_h_ctg.protein.fa	   README.txt
DK_0911_v04_p_ctg.EffectorP2.0.fa


In [10]:
###now get the annotations of h
effectorp2_h = [x.id for x in SeqIO.parse(os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911/Secretome',\
                                            'DK_0911_v04_h_ctg.EffectorP2.0.fa'),'fasta')]
effectorp2_h_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_h_ctg.effectorp20.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, effectorp2_h_bed, effectorp2_h)

secretome_h = [x.id for x in SeqIO.parse(os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911/Secretome',\
                                            'DK_0911_v04_h_ctg.protein.fa'),'fasta')]
secretome_h_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_h_ctg.secretome.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, secretome_h_bed, secretome_h)




In [12]:
!ls '/home/benjamin/genome_assembly/Warrior/DK0911_v04/protein_annotation/DK_0911_v04LT_p_ctg/combined'

DK_0911_v04LT_p_ctg.Annotation_tax_scope.tablist
DK_0911_v04LT_p_ctg.busco.tablist
DK_0911_v04LT_p_ctg.cat.tablist
DK_0911_v04LT_p_ctg.CDD.tablist
DK_0911_v04LT_p_ctg.COG.tablist
DK_0911_v04LT_p_ctg.Coils.tablist
DK_0911_v04LT_p_ctg.dbCAN.tablist
DK_0911_v04LT_p_ctg.Gene3D.tablist
DK_0911_v04LT_p_ctg.GO_combined.tablist
DK_0911_v04LT_p_ctg.Hamap.tablist
DK_0911_v04LT_p_ctg.iprscan.tablist
DK_0911_v04LT_p_ctg.KEGG_combined.tablist
DK_0911_v04LT_p_ctg.merops.tablist
DK_0911_v04LT_p_ctg.MobiDBLite.tablist
DK_0911_v04LT_p_ctg.OGs.tablist
DK_0911_v04LT_p_ctg.PANTHER.tablist
DK_0911_v04LT_p_ctg.Pathway_no_KEGG.tablist
DK_0911_v04LT_p_ctg.Pfam.tablist
DK_0911_v04LT_p_ctg.PIRSF.tablist
DK_0911_v04LT_p_ctg.predicted_gene_name.tablist
DK_0911_v04LT_p_ctg.PRINTS.tablist
DK_0911_v04LT_p_ctg.ProDom.tablist
DK_0911_v04LT_p_ctg.ProSitePatterns.tablist
DK_0911_v04LT_p_ctg.ProSiteProfiles.tablist
DK_0911_v04LT_p_ctg.SFLD.tablist
DK_0911_v04LT_p_ctg.SignalP3.tablist
DK_0911_v04LT_p_ctg.SignalP_EUK.tabli

In [33]:
###now get the annotations of h
effectorp2_h = [x.id for x in SeqIO.parse(os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911/Secretome',\
                                            'DK_0911_v04_h_ctg.EffectorP2.0.fa'),'fasta')]
effectorp2_h_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_h_ctg.effectorp20.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, effectorp2_h_bed, effectorp2_h)

secretome_h = [x.id for x in SeqIO.parse(os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911/Secretome',\
                                            'DK_0911_v04_h_ctg.protein.fa'),'fasta')]
secretome_h_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_h_ctg.secretome.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, secretome_h_bed, secretome_h)

#BUSCOs
busco_h = !cut -f1 {os.path.join(DK0911_annotation_folder_hcontigs, '../funannotate/parsed/annotations.busco.txt')}
busco_h_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_h_ctg.busco.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, busco_h_bed, busco_h)

In [34]:
###now get the annotations of p
effectorp2_p = [x.id for x in SeqIO.parse(os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911/Secretome',\
                                            'DK_0911_v04_p_ctg.EffectorP2.0.fa'),'fasta')]
effectorp2_p_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_p_ctg.effectorp20.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, effectorp2_p_bed, effectorp2_p)

secretome_p = [x.id for x in SeqIO.parse(os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911/Secretome',\
                                            'DK_0911_v04_p_ctg.protein.fa'),'fasta')]
secretome_p_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_p_ctg.secretome.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, secretome_p_bed, secretome_p)

#BUSCOs
busco_p = !cut -f1 {os.path.join(DK0911_annotation_folder_pcontigs, '../funannotate/parsed/annotations.busco.txt')}
busco_p_bed = os.path.join('/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/DK0911_annotations',\
                                'DK_0911_v04_p_ctg.busco.gene.bed')
gff_locus_tag_to_bed6(DK0911_GFF_file, busco_p_bed, busco_p)

In [ ]:
### TO DO get the annotation for the BUSCOs in PST104 sorted as well.